In [1]:
# Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Read in all DFs
dat = pd.read_csv("./data/processed_data.csv")
vax = pd.read_csv("./data/processed_vax.csv")
sym = pd.read_csv("./data/processed_symptoms.csv")

In [3]:
# Pull only information from vax DF which could be sensibly used given we need to get rid of duplicates
vax = pd.DataFrame(vax.groupby("vaersId")["covid"].apply(sum))

# We make a single column vax DF which represents number of Covid vaccines received
vax = vax.reset_index()
vax.columns = ["vaersId", "nCovidVax"]

In [4]:
# Merge all three dataframes together, combining X (features) and y (target)
sigma = dat.merge(vax, on="vaersId").merge(sym, on="vaersId")

In [5]:
# Write df to csv
sigma.to_csv("./data/combined.csv", index=False)